# Librerías utilizadas 

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import scatter_matrix
import plotly.express as px
#from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer)
#from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet#, LogisticRegression
%matplotlib inline

# Métrica de precisión
### Error logarítmico cuadrático medio

In [2]:
def rmsle(actual, predictions):
    log_diff = np.log((predictions+1)/(actual+1))
    return np.sqrt(np.mean(log_diff**2))

# Función de prueba de modelos

In [3]:
def mod_eval(xtrain,xtest,ytrain,ytest,regressor,parameters,score):
    preds=pd.DataFrame(index=list(range(1,25)))
    
    #gs = GridSearchCV(regressor, parameters, cv=5, n_jobs=1, verbose=1, scoring = score)
    gs = GridSearchCV(regressor, parameters, cv=5, n_jobs=1, scoring = score)
    gs.fit(xtrain, ytrain)
    
    mdl=gs.best_estimator_
    fctest=gs.predict(xtest)
    
    mdl_metric=gs.best_score_
    tst_metric=rmsle(ytest,fctest)
    
    return mdl,mdl_metric,tst_metric

# Lectura e inspección de datos

In [4]:
data = pd.read_csv("data/Train.csv")
data.info()

C:\Users\JJIMENEZG\AppData\Local\Temp\ipykernel_8620\1584083269.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/Train.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   401125 non-null  int64  
 1   SalePrice                 401125 non-null  int64  
 2   MachineID                 401125 non-null  int64  
 3   ModelID                   401125 non-null  int64  
 4   datasource                401125 non-null  int64  
 5   auctioneerID              380989 non-null  float64
 6   YearMade                  401125 non-null  int64  
 7   MachineHoursCurrentMeter  142765 non-null  float64
 8   UsageBand                 69639 non-null   object 
 9   saledate                  401125 non-null  object 
 10  fiModelDesc               401125 non-null  object 
 11  fiBaseModel               401125 non-null  object 
 12  fiSecondaryDesc           263934 non-null  object 
 13  fiModelSeries             56908 non-null   o

# Selección de features y limpieza de datos

In [5]:
data4preds= data[['YearMade','MachineHoursCurrentMeter','UsageBand','Transmission','ProductGroup','Drive_System','SalePrice']]
data4preds.head()
#scatter_matrix(data4preds, figsize=(15,9));

,YearMade,MachineHoursCurrentMeter,UsageBand,Transmission,ProductGroup,Drive_System,SalePrice
0,2004,68.0,Low,NaN,WL,NaN,66000
1,1996,4640.0,Low,NaN,WL,NaN,57000
2,2001,2838.0,High,NaN,SSL,NaN,10000
3,2001,3486.0,High,NaN,TEX,NaN,38500
4,2007,722.0,Medium,NaN,SSL,NaN,11000


In [6]:
data4preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   YearMade                  401125 non-null  int64  
 1   MachineHoursCurrentMeter  142765 non-null  float64
 2   UsageBand                 69639 non-null   object 
 3   Transmission              183230 non-null  object 
 4   ProductGroup              401125 non-null  object 
 5   Drive_System              104361 non-null  object 
 6   SalePrice                 401125 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 21.4+ MB


In [7]:
data4preds.describe().T

,count,mean,std,min,25%,50%,75%,max
YearMade,401125.0,1899.156901,291.797469,1000.0,1985.0,1995.0,2000.0,2013.0
MachineHoursCurrentMeter,142765.0,3457.955353,27590.256413,0.0,0.0,0.0,3025.0,2483300.0
SalePrice,401125.0,31099.712848,23036.898502,4750.0,14500.0,24000.0,40000.0,142000.0


In [8]:
data4preds['UsageBand'] =data4preds.UsageBand.map({'Low':1, 'High':3, 'Medium':2})
data4preds

C:\Users\JJIMENEZG\AppData\Local\Temp\ipykernel_8620\885841638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4preds['UsageBand'] =data4preds.UsageBand.map({'Low':1, 'High':3, 'Medium':2})


,YearMade,MachineHoursCurrentMeter,UsageBand,Transmission,ProductGroup,Drive_System,SalePrice
0,2004,68.0,1.0,NaN,WL,NaN,66000
1,1996,4640.0,1.0,NaN,WL,NaN,57000
2,2001,2838.0,3.0,NaN,SSL,NaN,10000
3,2001,3486.0,3.0,NaN,TEX,NaN,38500
4,2007,722.0,2.0,NaN,SSL,NaN,11000
...,...,...,...,...,...,...,...
401120,2005,NaN,NaN,NaN,TEX,NaN,10500
401121,2005,NaN,NaN,NaN,TEX,NaN,11000
401122,2005,NaN,NaN,NaN,TEX,NaN,11500
401123,2005,NaN,NaN,NaN,TEX,NaN,9000


In [9]:
data4preds=data4preds[data4preds['YearMade']>1960]
data4preds

,YearMade,MachineHoursCurrentMeter,UsageBand,Transmission,ProductGroup,Drive_System,SalePrice
0,2004,68.0,1.0,NaN,WL,NaN,66000
1,1996,4640.0,1.0,NaN,WL,NaN,57000
2,2001,2838.0,3.0,NaN,SSL,NaN,10000
3,2001,3486.0,3.0,NaN,TEX,NaN,38500
4,2007,722.0,2.0,NaN,SSL,NaN,11000
...,...,...,...,...,...,...,...
401120,2005,NaN,NaN,NaN,TEX,NaN,10500
401121,2005,NaN,NaN,NaN,TEX,NaN,11000
401122,2005,NaN,NaN,NaN,TEX,NaN,11500
401123,2005,NaN,NaN,NaN,TEX,NaN,9000


In [10]:
dummies = pd.get_dummies(data4preds[['Transmission','ProductGroup','Drive_System']])
data4preds=pd.concat([data4preds,dummies],axis=1).drop(['Transmission','ProductGroup','Drive_System','Transmission_AutoShift'],axis=1)
data4preds

,YearMade,MachineHoursCurrentMeter,UsageBand,SalePrice,Transmission_Autoshift,Transmission_Direct Drive,Transmission_Hydrostatic,Transmission_None or Unspecified,Transmission_Powershift,Transmission_Powershuttle,...,ProductGroup_BL,ProductGroup_MG,ProductGroup_SSL,ProductGroup_TEX,ProductGroup_TTT,ProductGroup_WL,Drive_System_All Wheel Drive,Drive_System_Four Wheel Drive,Drive_System_No,Drive_System_Two Wheel Drive
0,2004,68.0,1.0,66000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1996,4640.0,1.0,57000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2001,2838.0,3.0,10000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2001,3486.0,3.0,38500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2007,722.0,2.0,11000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,2005,NaN,NaN,10500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401121,2005,NaN,NaN,11000,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401122,2005,NaN,NaN,11500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401123,2005,NaN,NaN,9000,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
data4preds['UsageBand'].fillna(0,inplace=True)
data4preds

,YearMade,MachineHoursCurrentMeter,UsageBand,SalePrice,Transmission_Autoshift,Transmission_Direct Drive,Transmission_Hydrostatic,Transmission_None or Unspecified,Transmission_Powershift,Transmission_Powershuttle,...,ProductGroup_BL,ProductGroup_MG,ProductGroup_SSL,ProductGroup_TEX,ProductGroup_TTT,ProductGroup_WL,Drive_System_All Wheel Drive,Drive_System_Four Wheel Drive,Drive_System_No,Drive_System_Two Wheel Drive
0,2004,68.0,1.0,66000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1996,4640.0,1.0,57000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2001,2838.0,3.0,10000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2001,3486.0,3.0,38500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2007,722.0,2.0,11000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,2005,NaN,0.0,10500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401121,2005,NaN,0.0,11000,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401122,2005,NaN,0.0,11500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
401123,2005,NaN,0.0,9000,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
data4preds.dropna(inplace=True)
data4preds

,YearMade,MachineHoursCurrentMeter,UsageBand,SalePrice,Transmission_Autoshift,Transmission_Direct Drive,Transmission_Hydrostatic,Transmission_None or Unspecified,Transmission_Powershift,Transmission_Powershuttle,...,ProductGroup_BL,ProductGroup_MG,ProductGroup_SSL,ProductGroup_TEX,ProductGroup_TTT,ProductGroup_WL,Drive_System_All Wheel Drive,Drive_System_Four Wheel Drive,Drive_System_No,Drive_System_Two Wheel Drive
0,2004,68.0,1.0,66000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1996,4640.0,1.0,57000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2001,2838.0,3.0,10000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2001,3486.0,3.0,38500,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2007,722.0,2.0,11000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400463,2001,24.0,1.0,32200,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
400704,1999,24.0,1.0,55000,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
400843,2006,24.0,1.0,15200,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
400883,2008,24.0,1.0,13000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


# Conjunto de entrenamiento y prueba 

In [13]:
X_data = data4preds.drop('SalePrice',axis=1)
y_data = data4preds['SalePrice']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33)
score =make_scorer(rmsle,greater_is_better=False)

In [15]:
y_test

295235     46000
374360     46000
371815     10000
324704     55000
363227     77500
           ...  
387206     80000
299819     70000
388664    102500
374087     27500
370084     45000
Name: SalePrice, Length: 41648, dtype: int64

# Grid Search de modelos de regresión:
## Regresión lineal
## Lasso
## Ridge
## Elastic Net

#### Se prueban parámetros separados en escala  logarítmica para los últimos tres modelos.

In [16]:
mdl_list=[LinearRegression(),Lasso(),Ridge(),ElasticNet()]
param_array=list(np.logspace(-2,4,num=100))
mdl_params=[{},{'alpha':param_array},{'alpha':param_array},{'alpha':param_array,'l1_ratio':list(np.linspace(0.01,0.03,10))}]
best_mdl=[]
scores=[]
for m in range(0,len(mdl_list)):
    res=mod_eval(X_train,X_test,y_train,y_test,mdl_list[m],mdl_params[m],score)
    best_mdl.append(res[0])
    scores.append(res[2])

summary=pd.DataFrame(list(zip(best_mdl,scores)),columns=['Mejor Modelo','Scores'])

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.132e+13, tolerance: 4.195e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.368e+12, tolerance: 4.211e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value en

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.195e+12, tolerance: 4.211e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.814e+10, tolerance: 4.186e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value en

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.382e+09, tolerance: 4.186e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.754e+12, tolerance: 4.217e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value en

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.264e+12, tolerance: 4.217e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.037e+12, tolerance: 4.207e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value en

...

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid valu

In [17]:
summary

,Mejor Modelo,Scores
0,LinearRegression(),0.567405
1,Lasso(alpha=305.38555088334186),0.555447
2,Ridge(alpha=1232.8467394420659),0.559299
3,"ElasticNet(alpha=0.02009233002565047, l1_ratio...",0.554539


# Selección del mejor modelo con los datos de entrenamiento

In [18]:
best_mdl=summary[summary['Scores']==min(summary.Scores)].loc[:,'Mejor Modelo'].reset_index()
best_mdl

,index,Mejor Modelo
0,3,"ElasticNet(alpha=0.02009233002565047, l1_ratio..."


# Conjunto de datos de validación

In [20]:
feats4val = pd.read_csv('data/test.csv').set_index('SalesID')
y4val = pd.read_csv('data/test_actual.csv').set_index('SalesID')
#data4val = data4val.set_index('SalesID')
feats4val.info()
feats4val.describe().T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11573 entries, 1222837 to 6333349
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MachineID                 11573 non-null  int64  
 1   ModelID                   11573 non-null  int64  
 2   datasource                11573 non-null  int64  
 3   auctioneerID              11573 non-null  int64  
 4   YearMade                  11573 non-null  int64  
 5   MachineHoursCurrentMeter  4739 non-null   float64
 6   UsageBand                 4031 non-null   object 
 7   saledate                  11573 non-null  object 
 8   fiModelDesc               11573 non-null  object 
 9   fiBaseModel               11573 non-null  object 
 10  fiSecondaryDesc           8037 non-null   object 
 11  fiModelSeries             1759 non-null   object 
 12  fiModelDescriptor         2897 non-null   object 
 13  ProductSize               5743 non-null   object 
 14

,count,mean,std,min,25%,50%,75%,max
MachineID,11573.0,1.651495e+06,652248.533150,150.0,1067304.0,1862151.0,2270530.0,2485252.0
ModelID,11573.0,8.940136e+03,7807.393696,28.0,3362.0,4763.0,14303.0,37197.0
datasource,11573.0,1.526227e+02,14.872064,121.0,149.0,149.0,172.0,173.0
auctioneerID,11573.0,7.547481e+00,22.307077,0.0,1.0,1.0,3.0,99.0
YearMade,11573.0,1.895332e+03,305.481901,1000.0,1993.0,2001.0,2005.0,2014.0
MachineHoursCurrentMeter,4739.0,5.482141e+03,6391.097182,0.0,1268.0,3786.0,7793.0,89200.0


# Limpieza de datos de validación

In [21]:
data4val = pd.concat([feats4val[['YearMade','MachineHoursCurrentMeter','UsageBand','Transmission','ProductGroup','Drive_System']],
                   y4val['SalePrice']],axis=1)
data4val['UsageBand'] =data4val.UsageBand.map({'Low':1, 'High':3, 'Medium':2})
data4val=data4val[data4val['YearMade']>1960]
dummies = pd.get_dummies(data4val[['Transmission','ProductGroup','Drive_System']])
data4val=pd.concat([data4val,dummies],axis=1).drop(['Transmission','ProductGroup','Drive_System'],axis=1)
data4val['UsageBand'].fillna(0,inplace=True)
data4val.dropna(inplace=True)

In [22]:
X_val = data4val.drop('SalePrice',axis=1)
y_val = data4val['SalePrice']

# Predicción del modelo

In [23]:
mdl_val=best_mdl.loc[:,'Mejor Modelo'][0]
y_hat = mdl_val.predict(X_val)
rmsle(y_val, y_hat)

C:\Users\JJIMENEZG\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0.599499603598123